## Goal
- To demonstrate [PyReshaper](https://github.com/NCAR/PyReshaper)-like operation with xarray.

## Model

The [PyReshaper](https://github.com/NCAR/PyReshaper) converts multiple files that are in time-slice format (i.e., synoptic format) into files that are in time-series format.  

A **“time-slice”** is a file with multiple time-dependent variables in it, but only 1 (or a few) time steps.  That is, there will be a “time” coordinate variable in the file with length 1 (or a small number), and multiple variables in the file that depend upon the “time” dimension.  There may also be variables in the file that do not depend upon “time” at all!


To demonstrate this, we will generate some fake "time-slice" data that meets the above requirements.

## Import Packages

In [1]:
import xarray as xr
from dask.distributed import Client
import numpy as np 
import pandas as pd
import os

In [2]:
client = Client(processes=False)
client

Client Scheduler: inproc://128.117.11.17/27673/1 Dashboard: http://localhost:8787/status,Cluster Workers: 1 Cores: 8 Memory: 17.18 GB


## Generate Fake Data

In [3]:
# Create directory to save time slices data in
os.makedirs('data/timeslices', exist_ok=True)

In [4]:
def create_data_array(time, lat, lon, name):
    """Generate some random xarray dataarray"""
    data_array = xr.DataArray(np.random.randn(len([time]), len(lat), len(lon)), 
                      coords={'time': [time], 'lat': lat, 'lon': lon},
                      dims=('time', 'lat', 'lon'),
                      name=name)
    return data_array 

def generate_fake_data(time, suffix):
    """Create xarray 'time-slice' dataset and save it to disk"""
    # generate latitude and longitude values 
    lat = np.linspace(start=-90, stop=90, num=180, dtype='int')
    lon = np.linspace(start=-180, stop=180, num=360, dtype='int')
    
    # Create some variables 
    sst = create_data_array(time, lat, lon, name='sst')
    prec = create_data_array(time, lat, lon, name='prec')
    pressure = create_data_array(time, lat, lon, name='pressure')
    
    # Create some meta data variables. These variables are the same for all time slices
    meta = xr.DataArray(np.arange(len(lat)*len(lon)).reshape(len(lat), len(lon)), 
                        coords={'lat': lat, 'lon': lon}, 
                        dims=('lat', 'lon'),
                        name='meta_var')
    nlat = xr.DataArray(lat/2, coords={'lat': lat}, dims=('lat'))
    nlon = xr.DataArray(lon/2, coords={'lon': lon}, dims=('lon'))
    dset = xr.Dataset({'sst': sst, 'pressure': pressure, 'prec': prec, 'meta_var': meta, 'nlat': nlat, 'nlon': nlon})
    
    # Add some global attributes to our dataset 
    dset.attrs['created on'] = '2010-10-10'
    dset.attrs['created by'] = 'foo'
    dset.attrs['experiment_name'] = 'bar'
    path = f'data/timeslices/tslice{str(suffix)}.nc'
    dset.to_netcdf(path, engine='netcdf4', mode='w')

In [5]:
def generate_data(start_date='2000-01-01', freq='1M', periods=24):
    times = pd.DatetimeIndex(start=start_date, freq=freq, periods=periods)
    
    for index, time in enumerate(times):
        generate_fake_data(time, index)

In [6]:
%time generate_data()

CPU times: user 576 ms, sys: 65.7 ms, total: 642 ms
Wall time: 628 ms


In [7]:
!ncdump -h data/timeslices/tslice0.nc

netcdf tslice0 {
dimensions:
	time = 1 ;
	lat = 180 ;
	lon = 360 ;
variables:
	int64 time(time) ;
		time:units = "days since 2000-01-31 00:00:00" ;
		time:calendar = "proleptic_gregorian" ;
	int64 lat(lat) ;
	int64 lon(lon) ;
	double sst(time, lat, lon) ;
		sst:_FillValue = NaN ;
	double pressure(time, lat, lon) ;
		pressure:_FillValue = NaN ;
	double prec(time, lat, lon) ;
		prec:_FillValue = NaN ;
	int64 meta_var(lat, lon) ;
	double nlat(lat) ;
		nlat:_FillValue = NaN ;
	double nlon(lon) ;
		nlon:_FillValue = NaN ;

// global attributes:
		:created\ on = "2010-10-10" ;
		:created\ by = "foo" ;
		:experiment_name = "bar" ;
}


In [8]:
!du -s -h data/timeslices/

 48M	data/timeslices/


## From time-slice files to time-series files
The PyReshaper converts these time-slice files into time-series files by creating a separate file for each time-series variable.  Inside each time-series file exists the “time” variable, the corresponding time-series variable, and all of the metadata variables.


### Step 1: Open all time slices files with `xarray.open_mfdataset()`

In [9]:
dset = xr.open_mfdataset('data/timeslices/*.nc', data_vars='minimal')
dset

<xarray.Dataset>
Dimensions:   (lat: 180, lon: 360, time: 24)
Coordinates:
  * lat       (lat) int64 -90 -88 -87 -86 -85 -84 -83 ... 83 84 85 86 87 88 90
  * lon       (lon) int64 -180 -178 -177 -176 -175 -174 ... 175 176 177 178 180
  * time      (time) datetime64[ns] 2000-01-31 2000-02-29 ... 2000-10-31
Data variables:
    meta_var  (lat, lon) int64 0 1 2 3 4 5 ... 64795 64796 64797 64798 64799
    nlat      (lat) float64 -45.0 -44.0 -43.5 -43.0 ... 43.0 43.5 44.0 45.0
    nlon      (lon) float64 -90.0 -89.0 -88.5 -88.0 ... 88.0 88.5 89.0 90.0
    sst       (time, lat, lon) float64 dask.array<shape=(24, 180, 360), chunksize=(1, 180, 360)>
    pressure  (time, lat, lon) float64 dask.array<shape=(24, 180, 360), chunksize=(1, 180, 360)>
    prec      (time, lat, lon) float64 dask.array<shape=(24, 180, 360), chunksize=(1, 180, 360)>
Attributes:
    created on:       2010-10-10
    created by:       foo
    experiment_name:  bar

In [10]:
dset.meta_var

<xarray.DataArray 'meta_var' (lat: 180, lon: 360)>
array([[    0,     1,     2, ...,   357,   358,   359],
       [  360,   361,   362, ...,   717,   718,   719],
       [  720,   721,   722, ...,  1077,  1078,  1079],
       ...,
       [63720, 63721, 63722, ..., 64077, 64078, 64079],
       [64080, 64081, 64082, ..., 64437, 64438, 64439],
       [64440, 64441, 64442, ..., 64797, 64798, 64799]])
Coordinates:
  * lat      (lat) int64 -90 -88 -87 -86 -85 -84 -83 ... 83 84 85 86 87 88 90
  * lon      (lon) int64 -180 -178 -177 -176 -175 -174 ... 175 176 177 178 180

In [11]:
dset.sst

<xarray.DataArray 'sst' (time: 24, lat: 180, lon: 360)>
dask.array<shape=(24, 180, 360), dtype=float64, chunksize=(1, 180, 360)>
Coordinates:
  * lat      (lat) int64 -90 -88 -87 -86 -85 -84 -83 ... 83 84 85 86 87 88 90
  * lon      (lon) int64 -180 -178 -177 -176 -175 -174 ... 175 176 177 178 180
  * time     (time) datetime64[ns] 2000-01-31 2000-02-29 ... 2000-10-31

In [12]:
dset.data_vars

Data variables:
    meta_var  (lat, lon) int64 0 1 2 3 4 5 ... 64795 64796 64797 64798 64799
    nlat      (lat) float64 -45.0 -44.0 -43.5 -43.0 ... 43.0 43.5 44.0 45.0
    nlon      (lon) float64 -90.0 -89.0 -88.5 -88.0 ... 88.0 88.5 89.0 90.0
    sst       (time, lat, lon) float64 dask.array<shape=(24, 180, 360), chunksize=(1, 180, 360)>
    pressure  (time, lat, lon) float64 dask.array<shape=(24, 180, 360), chunksize=(1, 180, 360)>
    prec      (time, lat, lon) float64 dask.array<shape=(24, 180, 360), chunksize=(1, 180, 360)>

In [13]:
dset.dims

Frozen(SortedKeysDict({'lat': 180, 'lon': 360, 'time': 24}))

In [14]:
dset.coords

Coordinates:
  * lat      (lat) int64 -90 -88 -87 -86 -85 -84 -83 ... 83 84 85 86 87 88 90
  * lon      (lon) int64 -180 -178 -177 -176 -175 -174 ... 175 176 177 178 180
  * time     (time) datetime64[ns] 2000-01-31 2000-02-29 ... 2000-10-31

In [15]:
dset.attrs

OrderedDict([('created on', '2010-10-10'),
             ('created by', 'foo'),
             ('experiment_name', 'bar')])

### Step 2: Figure out which variables depend on time and which ones do not

In [16]:
all_variables = dset.data_vars.keys()
all_variables

KeysView(Data variables:
    meta_var  (lat, lon) int64 0 1 2 3 4 5 ... 64795 64796 64797 64798 64799
    nlat      (lat) float64 -45.0 -44.0 -43.5 -43.0 ... 43.0 43.5 44.0 45.0
    nlon      (lon) float64 -90.0 -89.0 -88.5 -88.0 ... 88.0 88.5 89.0 90.0
    sst       (time, lat, lon) float64 dask.array<shape=(24, 180, 360), chunksize=(1, 180, 360)>
    pressure  (time, lat, lon) float64 dask.array<shape=(24, 180, 360), chunksize=(1, 180, 360)>
    prec      (time, lat, lon) float64 dask.array<shape=(24, 180, 360), chunksize=(1, 180, 360)>)

In [17]:
tseries_vars = []
metadata_vars = []

for var in all_variables:
    if 'time' in dset.data_vars[var].dims:
        tseries_vars.append(var)
    else:
        metadata_vars.append(var)

In [18]:
tseries_vars

['sst', 'pressure', 'prec']

In [19]:
metadata_vars

['meta_var', 'nlat', 'nlon']

### Step 3: Convert time-slice dataset into time-series dataset
After figuring out which variables depend on time, we are able to do the conversion from time-slice to time-series. 

In the `create_time_series()` function below, we do the following:
- create a list of time-series `datasets`. Each element in this list contains one time-series variable, with metadata variables, and corresponding dataset global attributes. 
- create a list of `paths` to which to save each corresponding dataset.

This allows us to use `xarray.save_mfdataset()` function, which uses `dask` under the hood to write multiple datasets to disk as netCDF files simultaneously. 

In [20]:
def create_time_series(dset):
    datasets = []
    paths = []
    
    for vname in tseries_vars:
        # Get the data corresponding to this time series variable dataset
        var_dset = dset.data_vars[vname].to_dataset()
        
        # Add metadata variables to time series variable dataset
        for mvar in metadata_vars:
            var_dset[mvar] = dset.data_vars[mvar]
            
        # Add dataset global attributes to time series variable dataset
        var_dset.attrs = dset.attrs
        fpath = f'./data/timeseries/{vname}.nc'
        datasets.append(var_dset)
        paths.append(fpath)
    return datasets, paths

In [21]:
datasets, paths = create_time_series(dset)

In [22]:
datasets

[<xarray.Dataset>
 Dimensions:   (lat: 180, lon: 360, time: 24)
 Coordinates:
   * lat       (lat) int64 -90 -88 -87 -86 -85 -84 -83 ... 83 84 85 86 87 88 90
   * lon       (lon) int64 -180 -178 -177 -176 -175 -174 ... 175 176 177 178 180
   * time      (time) datetime64[ns] 2000-01-31 2000-02-29 ... 2000-10-31
 Data variables:
     sst       (time, lat, lon) float64 dask.array<shape=(24, 180, 360), chunksize=(1, 180, 360)>
     meta_var  (lat, lon) int64 0 1 2 3 4 5 ... 64795 64796 64797 64798 64799
     nlat      (lat) float64 -45.0 -44.0 -43.5 -43.0 ... 43.0 43.5 44.0 45.0
     nlon      (lon) float64 -90.0 -89.0 -88.5 -88.0 ... 88.0 88.5 89.0 90.0
 Attributes:
     created on:       2010-10-10
     created by:       foo
     experiment_name:  bar, <xarray.Dataset>
 Dimensions:   (lat: 180, lon: 360, time: 24)
 Coordinates:
   * lat       (lat) int64 -90 -88 -87 -86 -85 -84 -83 ... 83 84 85 86 87 88 90
   * lon       (lon) int64 -180 -178 -177 -176 -175 -174 ... 175 176 177 178 180


In [23]:
paths

['./data/timeseries/sst.nc',
 './data/timeseries/pressure.nc',
 './data/timeseries/prec.nc']

In [24]:
mkdir -p data/timeseries

In [25]:
%time xr.save_mfdataset(datasets, paths, mode='w', engine='netcdf4')

CPU times: user 822 ms, sys: 135 ms, total: 958 ms
Wall time: 916 ms


In [26]:
!ncdump -h data/timeseries/pressure.nc

netcdf pressure {
dimensions:
	lat = 180 ;
	lon = 360 ;
	time = 24 ;
variables:
	int64 lat(lat) ;
	int64 lon(lon) ;
	int64 time(time) ;
		time:units = "days since 2000-01-31 00:00:00" ;
		time:calendar = "proleptic_gregorian" ;
	double pressure(time, lat, lon) ;
		pressure:_FillValue = NaN ;
	int64 meta_var(lat, lon) ;
	double nlat(lat) ;
		nlat:_FillValue = NaN ;
	double nlon(lon) ;
		nlon:_FillValue = NaN ;

// global attributes:
		:created\ on = "2010-10-10" ;
		:created\ by = "foo" ;
		:experiment_name = "bar" ;
}


In [27]:
xr.open_dataset("data/timeseries/prec.nc")

<xarray.Dataset>
Dimensions:   (lat: 180, lon: 360, time: 24)
Coordinates:
  * lat       (lat) int64 -90 -88 -87 -86 -85 -84 -83 ... 83 84 85 86 87 88 90
  * lon       (lon) int64 -180 -178 -177 -176 -175 -174 ... 175 176 177 178 180
  * time      (time) datetime64[ns] 2000-01-31 2000-02-29 ... 2000-10-31
Data variables:
    prec      (time, lat, lon) float64 ...
    meta_var  (lat, lon) int64 ...
    nlat      (lat) float64 ...
    nlon      (lon) float64 ...
Attributes:
    created on:       2010-10-10
    created by:       foo
    experiment_name:  bar

In [28]:
ds = xr.open_mfdataset("data/timeseries/*.nc")

In [29]:
print('dataset size in GB {:0.2f}\n'.format(ds.nbytes / 1e9))

dataset size in GB 0.04



In [30]:
ds.info()

xarray.Dataset {
dimensions:
	lat = 180 ;
	lon = 360 ;
	time = 24 ;

variables:
	int64 lat(lat) ;
	int64 lon(lon) ;
	datetime64[ns] time(time) ;
	float64 prec(time, lat, lon) ;
	int64 meta_var(lat, lon) ;
	float64 nlat(lat) ;
	float64 nlon(lon) ;
	float64 pressure(time, lat, lon) ;
	float64 sst(time, lat, lon) ;

// global attributes:
	:created on = 2010-10-10 ;
	:created by = foo ;
	:experiment_name = bar ;
}

In [31]:
!du -s -h data/timeseries/

 38M	data/timeseries/


In [32]:
%load_ext version_information
%version_information dask, numpy, xarray, netcdf4

Software versions
Python 3.6.6 64bit [GCC 4.2.1 Compatible Apple LLVM 6.1.0 (clang-602.0.53)]
IPython 7.0.1
OS Darwin 17.7.0 x86_64 i386 64bit
dask 0.19.4
numpy 1.15.1
xarray 0.10.9
netcdf4 1.4.1
Thu Oct 18 14:26:05 2018 MDT